In [1]:
import torch
import torch.nn as nn
from ResNetHer import ResNetHer
from muxcnn.models.lightning import LitModel
from muxcnn.utils import load_params, load_img

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)


In [4]:
fn_chk = "./smooth_sep_bn.pt"

In [13]:
model = ResNetHer([1,1,1])#, hermite=True)#, activation=None)

model.to('cuda')
train_on_gpu=True

# Instantiate the model and Lightning module
criterion = nn.CrossEntropyLoss()

lit_model = LitModel.load_from_checkpoint(fn_chk, 
                                          model=model, 
                                          criterion=criterion)

In [14]:
state_dict = lit_model.model.state_dict()

In [19]:
print(state_dict.keys())

odict_keys(['hermitepn.gamma', 'hermitepn.beta', 'hermitepn.running_mean', 'hermitepn.running_var', 'hermitepn.running_mean2', 'hermitepn.running_var2', 'conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.conv2.weight', 'layer1.0.herPN1.gamma', 'layer1.0.herPN1.beta', 'layer1.0.herPN1.running_mean', 'layer1.0.herPN1.running_var', 'layer1.0.herPN1.running_mean2', 'layer1.0.herPN1.running_var2', 'layer1.0.herPN2.gamma', 'layer1.0.herPN2.beta', 'layer1.0.herPN2.running_mean', 'layer1.0.herPN2.running_var', 'layer1.0.herPN2.running_mean2', 'layer1.0.herPN2.running_var2', 'layer2.0.conv1.weight', 'layer2.0.conv2.weight', 'layer2.0.shortcut.0.weight', 'layer2.0.herPN1.gamma', 'layer2.0.herPN1.beta', 'layer2.0.herPN1.running_mean', 'layer2.0.herPN1.running_var', 'layer2.0.herPN1.running_mean2', 'layer2.0.herPN1.running_var2', 'layer2.0.herPN2.gamma', 'layer2.0.herPN2.beta', 'layer2.0.herPN2.running_mean

In [20]:
import torch
from muxcnn.models.ResNet20 import ResNet, BasicBlock

fn_chk2 = "./ResNet8.pt"
device="cpu"

model = ResNet(BasicBlock,[1,1,1])
model.eval() 

trained_param = torch.load(fn_chk2, map_location = torch.device(device))
if 'state_dict' in trained_param.keys():
    trained_param = trained_param['state_dict']
    print("[INFO] extracting state_dict from the checkpoint")

trained_param = {key : value.cpu() for key,value in trained_param.items()}
model.load_state_dict(trained_param)

<All keys matched successfully>

In [23]:
model.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.running_mean', 'layer2.0.bn2.running_var', 'layer2.0.bn2.num_batches_tracked', 'layer2.0.shortcut.0.weight', 'layer2.0.shortcut.1.running_mean', 'layer2.0.shortcut.1.running_var', 'layer2.0.shortcut.1.num_batches_tracked', 'layer3.0.conv1.weight', 'layer3.0.bn1.running_mean', 'layer3.0.bn1.running_var', 'layer3.0.bn1.num_batches_tracked', 'layer3.0.conv2.weight', 'layer3.0.bn2.running_mean', 'layer3.0.bn2.running_var', 'layer3.0.bn2.num_batches_tracked', 'layer3.0.shortcut.0.weight'

# Comapre MuxedHer and TorchHer

## Torch

In [5]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
model = ResNetHer([1,1,1])#, hermite=True)#, activation=None)
model.eval()
model.to('cpu')

# Instantiate the model and Lightning module
criterion = nn.CrossEntropyLoss()

lit_model = LitModel.load_from_checkpoint(fn_chk, 
                                          model=model, 
                                          criterion=criterion)

In [6]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [7]:
torch_result = lit_model(img_tensor)
torch_class = torch.argmax(torch_result)
print(f"[PyTorch] It's a {classes[torch_class]}")

[PyTorch] It's a dog


## Mux

In [8]:
from muxcnn.resnet_muxconvHer import ResNet_MuxConvHer

In [9]:
muxed_model = ResNet_MuxConvHer(model)

In [10]:
result = muxed_model(img_tensor)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2


TypeError: parMuxBN_separate() missing 2 required positional arguments: 'outs' and 'nslots'